# 1. Introduction to Dataset Exploration

In this section, we will:

Load the African Wildlife dataset.

Explore the dataset structure.

Visualize sample images with bounding boxes.

In [ ]:
# Install matplotlib and opencv-python
%pip install matplotlib opencv-python

# Import libraries
import os
import cv2
import matplotlib.pyplot as plt

# Define dataset paths
train_image_dir = "data/african_wildlife/train/images"
train_label_dir = "data/african_wildlife/train/labels"


### Function to visualize images with bounding boxes

In [ ]:
# Function to visualize images with bounding boxes
def plot_image_with_boxes(image_path, label_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib
    height, width, _ = image.shape
    
    with open(label_path, "r") as file:
        for line in file:
            class_id, x_center, y_center, w, h = map(float, line.split())
            # Convert YOLO format to pixel coordinates
            x_center *= width
            y_center *= height
            w *= width
            h *= height
            x_min = int(x_center - w / 2)
            y_min = int(y_center - h / 2)
            x_max = int(x_center + w / 2)
            y_max = int(y_center + h / 2)
            
            # Draw bounding box
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
    
    plt.imshow(image)
    plt.title(f"Image: {os.path.basename(image_path)}")
    plt.axis("off")
    plt.show()

### Visualize a few samples

In [ ]:
sample_images = os.listdir(train_image_dir)[:3]  # First 3 images
for img in sample_images:
    img_path = os.path.join(train_image_dir, img)
    label_path = os.path.join(train_label_dir, img.replace(".jpg", ".txt"))
    plot_image_with_boxes(img_path, label_path)

# 2. Dataset Verification

We need to ensure that:

Every image has a corresponding label file.

The annotations are correct.

In [ ]:
# Verify dataset integrity
def verify_dataset(image_dir, label_dir):
    images = set(os.listdir(image_dir))
    labels = set(os.listdir(label_dir))
    
    missing_labels = []
    for img in images:
        label_file = img.replace(".jpg", ".txt")
        if label_file not in labels:
            missing_labels.append(img)
    
    if missing_labels:
        print(f"Missing labels for {len(missing_labels)} images.")
    else:
        print("All images have corresponding labels.")

verify_dataset(train_image_dir, train_label_dir)

# 3. Dataset Preprocessing
In this section, we will:

Resize images to a consistent resolution (e.g., 640x640).

Normalize pixel values.

In [ ]:
# Resize images and save to a new directory
def resize_images(image_dir, output_dir, size=(640, 640)):
    os.makedirs(output_dir, exist_ok=True)
    for img in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img)
        image = cv2.imread(img_path)
        resized_image = cv2.resize(image, size)
        cv2.imwrite(os.path.join(output_dir, img), resized_image)

# Resize training images
resize_images(train_image_dir, "data/african_wildlife/train/images_resized")

### train

In [ ]:
# Install ultralytics package
%pip install ultralytics

from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
model = YOLO("yolov8n.pt")  # Use 'yolov8s.pt' or 'yolov8m.pt' for larger models

# Train the model
results = model.train(data="data.yaml", epochs=50, imgsz=640, batch=16, name="african_wildlife_yolov8")

# 4. Model Training
In this section, we will:

Load the preprocessed dataset.

Train a YOLOv8 model on the African Wildlife dataset.

# 5. Model Evaluation
In this section, we will:

Evaluate the trained model on the validation set.

Compute key metrics like mAP, precision, and recall.

In [ ]:
from ultralytics import YOLO

# Load the pre-trained YOLOv8 model
model = YOLO("C:/Users/mophi/Desktop/wildlife-conservation-system/yolov8n.pt")
# Evaluate on the validation set
metrics = model.val()  # Use 'best_model.val()' if you loaded 'best.pt'

# Print key metrics
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")
print(f"Precision: {metrics.box.precision}")
print(f"Recall: {metrics.box.recall}")

# 6. Testing and Deployment
In this section, we will:

Test the model on new images.

Export the model for deployment.

In [ ]:
# Test on new images
test_images = ["path/to/test_image1.jpg", "path/to/test_image2.jpg"]  # Add your test images here

for img_path in test_images:
    best_model = YOLO("yolov8n.pt")
    results = best_model(img_path)
    for result in results:
        result.show()  
        print(f"Detected objects: {result.boxes.cls}")  

# Export the model to ONNX format
best_model.export(format="onnx")
print("Model exported to ONNX format for deployment.")